
Import Necessary Libraries


In [64]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient
import os

Data Extraction

In [49]:
try:
    data = pd.read_excel(r'Dataset\Raw Data\Amazing food dataset..xlsx')
    print('Data Extracted successfully')
except Exception as e:
    print (f'an error occured: {e}')

Data Extracted successfully


In [ ]:
# Drop duplicates values

data.drop_duplicates(inplace=True)

Data Cleaning and Transformation 


In [39]:
# Check for missing value
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             900 non-null    float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        900 non-null    object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [ ]:
# To Handle the misiing values

data = data.fillna({
    'Temperature': data['Temperature'].mean(),
    'CustomerFeedback': 'Unknown',
    'CustomerEmail': 'Unknown',
    'Staff_Email': 'Unknown'
})

In [ ]:
# Rename Columns

data = data.rename(columns ={
    'ProductName' : 'Product_Name',
    'UnitPrice' : 'Unit_Price',
    'StoreLocation' : 'Store_Location',
    'StaffPerformanceRating' : 'Staff_Performance_Rating',
    'CustomerFeedback' : 'Customer_Feedback',
    'DeliveryTime_min' : 'Delivery_Time_min',
    'OrderType' : 'Order_Type',
    'CustomerName' : 'Customer_Name',
    'CustomerAddress' : 'Customer_Address',
    'CustomerEmail' : 'Customer_Email',
    'DayOfWeek' : 'Day_Of_Week',
    'TotalSales' : 'Total_Sales',
    'PaymentType': 'Payment_Type',
    'PromotionApplied' : 'Promotion_Applied'
    })

In [ ]:
# save clean data to local machine

data.to_csv(r'Dataset\Cleaned Data\Clean data.csv', index=False)

Transformation

In [ ]:
# Create the product table

products = data[['Product_Name', 'Unit_Price']].drop_duplicates().reset_index(drop=True)
products['Product_id'] = np.random.randint(10000, 99999, size=len(products))
products = products[['Product_id', 'Product_Name', 'Unit_Price']]

In [ ]:
# Create customers table

customers = data[['Customer_Name', 'Customer_Address', 'Customer_PhoneNumber', 'Customer_Email', 'Customer_Feedback']].drop_duplicates().reset_index(drop=True)
customers ['Customer_id'] = np.random.randint(1000, 9999, size = len(customers))
customers = customers[['Customer_id','Customer_Name', 'Customer_Address', 'Customer_PhoneNumber', 'Customer_Email', 'Customer_Feedback']]

In [ ]:
# Create dates table

dates = data[['Date','Day_Of_Week' ]].drop_duplicates().reset_index(drop=True)
dates['Date_id'] = np.random.randint(10000,99999, size=len(dates)) 
dates = dates[['Date_id', 'Date','Day_Of_Week']]

In [ ]:
# Create staff table

staff = data [['Staff_Name', 'Staff_Email','Staff_Performance_Rating']].drop_duplicates().reset_index(drop=True)
staff ['staff_id'] = np.random.randint(10000, 99999, size=len(staff))
staff = staff [['staff_id', 'Staff_Name', 'Staff_Email','Staff_Performance_Rating']]

In [ ]:
# Fact table

fact_table = data.merge(customers, on=['Customer_Name', 'Customer_Address', 'Customer_PhoneNumber', 'Customer_Email', 'Customer_Feedback'], how='left')\
                .merge(products, on=['Product_Name', 'Unit_Price'], how='left')\
                    .merge(dates, on=['Date','Day_Of_Week' ], how='left')\
                        .merge(staff, on=['Staff_Name', 'Staff_Email','Staff_Performance_Rating'], how='left')
                        
fact_table= fact_table[['Customer_id', 'Product_id', 'Date_id', 'staff_id','Quantity','Store_Location',
                        'Payment_Type', 'Promotion_Applied', 'Weather', 'Temperature', 'Delivery_Time_min',
                        'Order_Type', 'Total_Sales']]

In [ ]:
# Save tables to local machine
products.to_csv(r'Dataset\Cleaned Data\Products data.csv', index=False)
customers.to_csv(r'Dataset\Cleaned Data\Customers data.csv', index=False)
staff.to_csv(r'Dataset\Cleaned Data\Staff data.csv',index=False)
dates.to_csv(r'Dataset\Cleaned Data\Dates data.csv', index=False)
fact_table.to_csv(r'Dataset\Cleaned Data\Fact table.csv', index=False)

Loading data into Azure Blob Storage

In [104]:
# Loading the environment variables
load_dotenv(override=True)

connect_str = os.getenv('Connect_str')
container_name = os.getenv('Container_name')

In [107]:
# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

# Loading data to Azure Blob Storage
files = [
    (data, r'Cleaned Data\clean data.csv' ),
    (products, r'Cleaned Data\Products data.csv'),
    (customers, r'Cleaned Data\Customers data.csv'),
    (staff, r'Cleaned Data\Staff data.csv'),
    (dates, r'Cleaned Data\Dates data.csv'),
    (fact_table, r'Cleaned Data\Fact table.csv')
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into Azure Blob Storage')

Cleaned Data\clean data.csv loaded into Azure Blob Storage
Cleaned Data\Products data.csv loaded into Azure Blob Storage
Cleaned Data\Customers data.csv loaded into Azure Blob Storage
Cleaned Data\Staff data.csv loaded into Azure Blob Storage
Cleaned Data\Dates data.csv loaded into Azure Blob Storage
Cleaned Data\Fact table.csv loaded into Azure Blob Storage
